In [2]:
from langchain_community.document_loaders import PDFPlumberLoader, PDFMinerLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory,FileChatMessageHistory
from dotenv import load_dotenv
import pandas as pd
pd.options.display.max_rows = 4000
import warnings
warnings.filterwarnings("ignore")


ModuleNotFoundError: No module named 'pwd'

In [ ]:
load_dotenv()

True

In [ ]:
embeddings_model = OpenAIEmbeddings()
chat_bot = ChatOpenAI()


In [ ]:
# loader1 = PDFMinerLoader("BANK_HOME_LOAN.pdf")
# pages1 = loader1.load()
loader = PDFPlumberLoader("BANK_HOME_LOAN.pdf")
pages = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=3000,
    chunk_overlap=400,
    length_function=len,
    is_separator_regex=False,
)


In [ ]:
text_chunks = text_splitter.split_documents(pages)

In [ ]:
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain_community.vectorstores import FAISS

In [ ]:
store = LocalFileStore('./cache/')
cache_embed = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings_model,
    document_embedding_cache = store,
    namespace=embeddings_model.model
    )

In [ ]:

db = FAISS.from_documents(text_chunks,cache_embed)

In [ ]:
chat_prompt = ChatPromptTemplate(
    input_variables = ['content','question','memory_messages', 'human_input'],
    messages=[
        MessagesPlaceholder(variable_name='memory_messages'),
        HumanMessagePromptTemplate.from_template(
            '''
            You are helpful analyst chatbot interacting with human,
            Answer the question by analysing the {content}
            Human question {question}
            {human_input}
            '''
        )
    ]
)

In [ ]:
chat_memory = ConversationBufferMemory(
    memory_key='memory_messages',
    input_key="human_input",
    return_messages=True)

In [ ]:
chatbot_chain = LLMChain(
     llm=chat_bot,
     prompt= chat_prompt,
     memory=chat_memory,
)

In [ ]:
while True:
    question = input('>>')
    content_db = db.similarity_search(question)
    result = chatbot_chain({'content':str(content_db[0].page_content),'question':question, 'human_input':''})
    print(result['text'])
    



The best bank for availing a home loan according to the evaluation conducted by the Financial Desk is not mentioned in the given text. However, it is stated that out of the 12 evaluated banks, 9 are in the government sector and 3 are in the private banking sector. This information can be useful for consumers who are looking to choose a bank for their home loan needs.


KeyboardInterrupt: 

In [ ]:
pip freeze > requirements.txt